In [ ]:
import numpy as np
import cupy as cp
from PIL import Image, ImageFilter
import time
from datetime import timedelta
from multiprocessing import Pool
import numba as nb
import GPUtil

def gpu_ram():
  return int(GPUtil.getGPUs()[0].memoryFree * 1_000_000)

@nb.njit(parallel=True)
def grid(buddha, n, m):
    for k in nb.prange(len(n)):
        buddha[n[k],m[k]] += 1
    return buddha

start = time.monotonic()
maxiter = 120
RAM = int(gpu_ram() / 8)
batchsize = int(RAM / 12)
number_of_batches = 1000
print('batchsize: {}\nmaxiter: {}\nnumber_of_batches: {}\ntotal: {}\n'.format(batchsize, maxiter, number_of_batches, number_of_batches * batchsize))

w, h = 3600, 3000 - 1
r, R, i, I = -2, 1, -1.25, 1.25
dr = (R - r) / w
di = (i - I) / h

threshold = 4
threshold_square = threshold * threshold
buddha = np.zeros((h, w), dtype=np.uint64)

for _ in range(number_of_batches):
    print('working on batch {}...'.format(_ + 1))
    cp.random.seed(_)
    c = cp.exp(1j * cp.random.random(batchsize) * 2 * cp.pi) * cp.sqrt(cp.random.random(batchsize)) * threshold
    c = c[cp.abs(1 - cp.sqrt(1 - 4 * c)) >= 1]
    z = cp.copy(c)
    not_diverging = cp.ones_like(z, dtype=bool)
    for _ in range(maxiter):
        z[not_diverging] = z[not_diverging] * z[not_diverging] + c[not_diverging]
        not_diverging = (z * cp.conj(z) < threshold_square)
    del z
    c = c[not_diverging == False]
    del not_diverging
    z = cp.zeros_like(c)
    for k in range(1, maxiter):
        z = z * z + c
        inside = (r <= z.real) * (z.real < R) * (i <= z.imag) * (z.imag < I)
        n = cp.floor((i - z[inside].imag) / di).get().astype(int)
        m = cp.floor((z[inside].real - r) / dr).get().astype(int)
        del inside
        buddha = grid(buddha, n, m)
        del m, n
        not_diverged = (z * np.conj(z) < threshold_square)
        z, c = z[not_diverged], c[not_diverged]
        del not_diverged
    del z, c

np.save('arrays/maxiter{}_batches{}_{}x{}.npy'.format(maxiter, number_of_batches, w, h), buddha)
buddha = buddha + buddha[::-1]
print('\ntime: {}'.format(timedelta(seconds=time.monotonic() - start)))

In [25]:
image = np.copy(buddha).T
m, M = np.min(image), np.max(image)
image = (image - m) / (M - m)

def transform(image):
    d = 0.5
    D = (d / (1 + d)) ** (1/2)
    high = 0.8
    image = (((image + d) / (1 + d)) ** (1/2) - D) / (1 - D)
    image = np.minimum(image / high, 1)
    return image

#image = transform(image)
image = 255 * image
image = Image.fromarray(image).convert('L')
#image = image.filter(ImageFilter.BoxBlur(1))
image.save('images/maxiter{}_batches{}_{}x{}.png'.format(maxiter, number_of_batches, w, h))